In [1]:
import os, random, sys, time, shutil, re, csv, datetime, pickle, math 
from datetime import date

import pandas as pd

from random import seed
from random import randrange, uniform

In [2]:
from bs4 import BeautifulSoup
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.webdriver import Options

In [3]:
chrome_options = Options()
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument("--log-level=3")  # fatal
chrome_options.add_argument("--start-maximized")
chrome_options.add_argument("--incognito")

In [4]:
accounts_path ='CSVs/commons/accounts.csv'

In [5]:
def get_accounts_df():
    try: return pd.read_csv(accounts_path)
    except: return pd.DataFrame()

In [6]:
def get_active_accounts():
    df = get_accounts_df()
    df = df[df.status == 'active']
    return df
get_active_accounts()

,updated_at,created_at,password,source,status,username,updated_at
0,02/03/2021,02/03/2021,dextra#2021,linkedin,active,zeipf4@gmail.com,03/03/2021
1,02/03/2021,02/03/2021,dextra#2021,linkedin,active,zeipf5@gmail.com,03/03/2021


In [7]:
browser = webdriver.Chrome(ChromeDriverManager().install(), options=chrome_options)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 88.0.4324
[WDM] - Get LATEST driver version for 88.0.4324
[WDM] - Driver [/home/carlos.cayres/.wdm/drivers/chromedriver/linux64/88.0.4324.96/chromedriver] found in cache


In [8]:
def auth_linkedin(browser, username, password):

    try:
        browser.get("https://www.linkedin.com/uas/login")
        time.sleep(uniform(2, 4))
        
        print('Authenticate for: ', username)
        username_field = browser.find_element_by_id('username')
        username_field.send_keys(username)
        time.sleep(uniform(2, 4))
    
        password_field = browser.find_element_by_id('password')
        password_field.send_keys(password)
        time.sleep(uniform(2, 4))
    
        password_field.submit()
        time.sleep(uniform(2, 4))
    except:
        print('ERROR: AUTH LINKEDIN FAILS')

In [9]:
def is_ban_page(browser):
    time.sleep(uniform(1, 2))
    return 'login-submit' in browser.current_url  

In [10]:
def is_home_page(browser):
    time.sleep(uniform(1, 2))
    soup = BeautifulSoup(browser.page_source, features='html.parser')
    user_div = soup.find('div', attrs={'class': 'scaffold-layout__sidebar'})
    return user_div is not None

In [11]:
def is_login_page(browser):
    time.sleep(uniform(1, 2))
    return 'uas/login' in browser.current_url

In [12]:
def account_status_for_current_page(browser):
    if is_ban_page(browser): return 'banned'
    elif is_home_page(browser): return 'active'
    elif is_login_page(browser): return 'reauth'
    else: return 'unknow'

In [13]:
def validate_accounts():
    df = get_accounts_df()
    
    for i, account in df.iterrows():
        
        time.sleep(uniform(1, 2))
        
        print('Open browser')
        browser = webdriver.Chrome(ChromeDriverManager().install(), options=chrome_options)
        
        print('Accessing linkedin')        
        time.sleep(uniform(1, 2))
        
        auth_linkedin(browser, account['username'], account['password']) 
        df.at[i,'status'] = account_status_for_current_page(browser)
        df.at[i,'updated_at'] = date.today().strftime("%d/%m/%Y")
        df.to_csv(accounts_path, index=False)
        
        browser.close()
        
    return df

In [14]:
def register_linkedin(username, password):
    df = get_accounts_df()
    data = { 'username': username,
             'password': password,
             'source': 'linkedin',
             'created_at': date.today().strftime("%d/%m/%Y"),
            ' updated_at': date.today().strftime("%d/%m/%Y"),
             'status': 'registred' }
    
    df = df.append(data, ignore_index=True)
    df = df.drop_duplicates(subset=['username'])
    df.to_csv(accounts_path, index=False)

In [15]:
validate_accounts()

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 88.0.4324
[WDM] - Get LATEST driver version for 88.0.4324
[WDM] - Driver [/home/carlos.cayres/.wdm/drivers/chromedriver/linux64/88.0.4324.96/chromedriver] found in cache


Open browser


Accessing linkedin
Authenticate for:  zeipf4@gmail.com


[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 88.0.4324
[WDM] - Get LATEST driver version for 88.0.4324
[WDM] - Driver [/home/carlos.cayres/.wdm/drivers/chromedriver/linux64/88.0.4324.96/chromedriver] found in cache


Open browser


Accessing linkedin
Authenticate for:  zeipf5@gmail.com


,updated_at,created_at,password,source,status,username,updated_at
0,02/03/2021,02/03/2021,dextra#2021,linkedin,unknow,zeipf4@gmail.com,04/03/2021
1,02/03/2021,02/03/2021,dextra#2021,linkedin,banned,zeipf5@gmail.com,04/03/2021


In [18]:
get_accounts_df()

,updated_at,created_at,password,source,status,username,updated_at
0,02/03/2021,02/03/2021,dextra#2021,linkedin,unknow,zeipf4@gmail.com,04/03/2021
1,02/03/2021,02/03/2021,dextra#2021,linkedin,banned,zeipf5@gmail.com,04/03/2021


In [17]:
#register_linkedin('zeipf5@gmail.com', 'dextra#2021')